In [1]:
import os
import sys
from pathlib import Path

# VSCode에서만 사용 가능한 건지 확인 필요
FILE = Path(__vsc_ipynb_file__).resolve()
ROOT = FILE.parents[1]
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))


In [55]:
import yaml

MODEL_FILE = '/mnt/hdd01/hyeongdo/workspace/yolov9/models/detect/yolov9.yaml'
with open(MODEL_FILE, encoding='ascii', errors='ignore') as f:
    model_yaml = yaml.safe_load(f)

print(type(model_yaml))
print(model_yaml.keys())

<class 'dict'>
dict_keys(['nc', 'depth_multiple', 'width_multiple', 'activation', 'anchors', 'backbone', 'head'])


https://tibetsandfox.tistory.com/27

In [66]:
import contextlib
m = model_yaml['head']
# print(len(m))
model_yaml['head']
# model_yaml['backbone'].__next__ # AttributeError: 'list' object has no attribute '__next__'
model_iter = iter(model_yaml['head'])
# print(model_iter.__next__())
# print(next(model_iter))
layers = []
for layer in model_iter:
    layers.append(layer)
print(f'number of layers: {len(layers)}')
dbg_layer_idx = len(layers)-1
print(f'index({dbg_layer_idx}) of layer args: {layers[dbg_layer_idx]}')
f, n, m, args = layers[dbg_layer_idx]

for j, a in enumerate(args):
    # print(j, a)
    # print(args[j])
    with contextlib.suppress(NameError):
        args[j] = eval(a) if isinstance(a, str) else a
    # print(args[j])

print(*args[1:])

# CBLinear
out_channel = args[0]
in_channel = 3
args = [in_channel, out_channel, *args[1:]]
# print(args)
print(*args)

number of layers: 29
index(28) of layer args: [[31, 34, 37, 16, 19, 22], 1, 'DualDDetect', ['nc']]

3 nc


In [3]:
from models.yolo import parse_model


In [5]:
input_channel = [3]

model, saved = parse_model(model_yaml, input_channel)


                 from  n    params  module                                  arguments                     
  0                -1  1         0  models.common.Silence                   []                            
  1                -1  1      1856  models.common.Conv                      [3, 64, 3, 2]                 
  2                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  3                -1  1    252160  models.common.RepNCSPELAN4              [128, 256, 128, 64, 2]        
  4                -1  1    164352  models.common.ADown                     [256, 256]                    
  5                -1  1   1004032  models.common.RepNCSPELAN4              [256, 512, 256, 128, 2]       
  6                -1  1    656384  models.common.ADown                     [512, 512]                    
  7                -1  1   4006912  models.common.RepNCSPELAN4              [512, 1024, 512, 256, 2]      
  8                -1  1   2623488  

In [23]:
print(type(model))

<class 'torch.nn.modules.container.Sequential'>


모델 정의 파일에서 마지막 detect 리스트의 첫 번째 원소를 가져와서 이전 레이어가 아닌 경우에 대한 처리

In [24]:
# 모델의 마지막 원소는 Head이다.
m = model[-1] # DualDDetect
# print(m)
# 가져온 Head block 모듈이 이전 레이어에 연결되어 있는 레이어가 아니면 
# # detect
# [[35, 38, 41], 1, DDetect, [nc]],  # Detect(P3, P4, P5)
print(m.f) # layer(s) from previous layer


[35, 38, 41]


모듈의 stride를 계산하기 위해 forward를 한 번 호출해서 계산하는데, 왜 그렇게 하는지 이유는 모르겠음

모델 구조가 정의되어 있는 yaml 파일을 읽어서 dict형의 자료로 변환한다.

`models/yolo.py`

```python
class DetectionModel(BaseModel):
    # model, input channels, number of classes
    def __init__(self, cfg='yolo.yaml', ch=3, nc=None, anchors=None):
        ...
        if isinstance(cfg, dict):
            self.yaml = cfg  # model dict
        else:  # is *.yaml
            import yaml  # for torch hub
            self.yaml_file = Path(cfg).name
            with open(cfg, encoding='ascii', errors='ignore') as f:
                self.yaml = yaml.safe_load(f)  # model dict
        ...
````

모델이 정의되어 있는 yaml 파일에 입력 레이어의 채널이 정의되어 있지 않으면 생성자로 전달한 채널 크기를 입력 레이어의 채널로 설정하고 그렇지 않으면 yaml 파일에 정의되어 있는 채널 크기를 입력 레이어의 채널러 설정한다.

```python
        ch = self.yaml['ch'] = self.yaml.get('ch', ch)  # input channels
```

모델 정의 파일에 정의된 클래스 수(`nc`)와 인스턴스 생성에서 전달한 클래스 수(`nc`)가 다르면
```python
 self.yaml['nc']
```
 를 인스턴스 생성에서 전달받은 값으로 설정한다

인스턴스 생성에서 `anchors` 값을 전달 받았으며 `self.yaml['anchor']`의 값을 전달 받은 값으로 변경한다.

`train.py(102-117)`

```python
# hyp is path/to/hyp.yaml or hyp dictionary
def train(hyp, opt, device, callbacks):
    ...
    check_suffix(weights, '.pt')  # check weights
    pretrained = weights.endswith('.pt')
    if pretrained:
        with torch_distributed_zero_first(LOCAL_RANK):
            weights = attempt_download(weights)  # download if not found locally
        ckpt = torch.load(weights, map_location='cpu')  # load checkpoint to CPU to avoid CUDA memory leak
        model = Model(cfg or ckpt['model'].yaml, ch=3, nc=nc, anchors=hyp.get('anchors')).to(device)  # create
        exclude = ['anchor'] if (cfg or hyp.get('anchors')) and not resume else []  # exclude keys
        csd = ckpt['model'].float().state_dict()  # checkpoint state_dict as FP32
        csd = intersect_dicts(csd, model.state_dict(), exclude=exclude)  # intersect
        model.load_state_dict(csd, strict=False)  # load
        LOGGER.info(f'Transferred {len(csd)}/{len(model.state_dict())} items from {weights}')  # report
    else:
        model = Model(cfg, ch=3, nc=nc, anchors=hyp.get('anchors')).to(device)  # create
    amp = check_amp(model)  # check AMP
    ...
```

`Model`은 `DetectionModel`이다. (`models/yolo.py`)

`train.py(564-567)`

```python
def main(opt, callbacks=Callbacks()):
    ...
        with open(opt.hyp, errors='ignore') as f:
            hyp = yaml.safe_load(f)  # load hyps dict
            if 'anchors' not in hyp:  # anchors commented in hyp.yaml
                hyp['anchors'] = 3
    ...
```

evolove 모드에서 hyper-parameter 정의 파일에 anchorsr가 정의 되어 있지 않으면 기본값 3으로 설정한다. 일반 train에서는 파일 설정 원형 그대로 전달한다.

만약 hyper-parameter 파일에 anchors 키가 선언되어 있고 값이 설정되어 있으면 모델 정의 yaml 파일에 정의되어 있는 anchors의 값이 이 값으로 대체 된다.

`models/yolo.py(592-599)`

```python
class DetectionModel(BaseModel):
    # model, input channels, number of classes
    def __init__(self, cfg='yolo.yaml', ch=3, nc=None, anchors=None):
        ...
        if anchors:
            LOGGER.info(f'Overriding model.yaml anchors with anchors={anchors}')
            self.yaml['anchors'] = round(anchors)  # override yaml value
        ...
```

In [35]:
from pprint import pprint

HYP_FILE = '/mnt/hdd01/hyeongdo/workspace/yolov9/data/hyps/hyp.scratch-high.yaml'
with open(HYP_FILE, errors='ignore') as f:
    hyp = yaml.safe_load(f)
# pprint(hyp)
print(hyp.get('anchors'))


None


number(n)는 네트워크의 depth gain을 계산할 때 사용한다.

`yolo.py (713-)`

```python
def parse_model(d, ch):  # model_dict, input_channels(3)
    ...
    for i, (f, n, m, args) in enumerate(d['backbone'] + d['head']):  # from, number, module, args
    m = eval(m) if isinstance(m, str) else m  # eval strings
    for j, a in enumerate(args):
        with contextlib.suppress(NameError):
            args[j] = eval(a) if isinstance(a, str) else a  # eval strings
    ...
```